In [ ]:
# 이 코랩 파일은 '학교 안내챗봇 만들기' 프로젝트 중 파인튜닝된 Gemma2b 모델을 gradio와 연결해서 챗봇 ui를 구성하는 부분입니다.
# 프로젝트 폴더 중 Test 폴더 안 여러 코랩 파일에서 개발 및 테스트를 거치고, 최종적으로 완성된 코드만 옮겨왔습니다.

In [1]:
!pip install transformers
!pip install gradio
!pip install accelerate
!pip install torch
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.0 MB

In [2]:
!huggingface-cli login # 허깅페이스 로그인
  # 개인 huggingface token을 입력


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

# basic Ver

In [ ]:
# 6/7 테스트 성공 (영어, 한국어)

# import
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import gradio as gr

# 허깅페이스 API 토큰 지정
hf_token = '{token}' # 이곳에 허깅페이스 토큰을 삽입

# 모델 지정 및 데이터타입 설정 (16bit)
model_id = "taeseo06/SchoolHelperChatbot-Gemma2B-Finetuning" # model: https://huggingface.co/taeseo06/SchoolHelperChatbot-Gemma2B-Finetuning
dtype = torch.bfloat16

# 채팅 함수
def gemma_chat(message, history):
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        use_auth_token=hf_token,
        torch_dtype=dtype,
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    chat = [
        { "role": "user", "content": message },
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs, max_new_tokens=2048)

    response = tokenizer.decode(outputs[0])

    # 응답을 정리
    response_cleaned = re.split("model", response)

    # 응답 반환
    return response_cleaned[1]

# Gradio 인터페이스 생성 및 실행
gr.ChatInterface(gemma_chat).launch(debug=True)

# systemPrompt Ver

In [ ]:
# import
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import gradio as gr

# 허깅페이스 API 토큰 지정
hf_token = '{token}' # 이곳에 허깅페이스 토큰을 삽입

# 모델 지정 및 데이터타입 설정 (16bit)
model_id = "taeseo06/SchoolHelperChatbot-Gemma2B-Finetuning" # model: https://huggingface.co/taeseo06/SchoolHelperChatbot-Gemma2B-Finetuning
dtype = torch.bfloat16

def clear_fn():
  return None

# 채팅 함수
def gemma_chat(message, history, system_prompt):
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        use_auth_token=hf_token,
        torch_dtype=dtype,
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if (system_prompt == ''):
      prompt_message = (message).strip() # system prompt가 비어있을 때 -> main_input만 prompt로 사용
    else:
      prompt_message = (system_prompt + "\n" + message).strip() # 사용자가 입력한 input 텍스트와 system_prompt 결합

    chat = [
        { "role": "user", "content": prompt_message },
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs, max_new_tokens=2048)

    response = tokenizer.decode(outputs[0])

    # 응답을 정리
    response_cleaned = re.split("model", response)

    # 응답 반환
    return response_cleaned[1]

# Gradio 인터페이스 생성 및 실행
with gr.Blocks() as demo:
    gr.HTML("""<center><h1 style='font-size:xx-large;'>학교생활 안내챗봇</h1></center>""")
    gr.HTML("""<center><b>base model: Google-Gemma-2b</b></center>""")


    chatbox = gr.Chatbot(label="Chat History")
    system_prompt = gr.Textbox(label="System Prompt", placeholder="system prompt 입력 (옵션)")
    main_input = gr.Textbox(label="Your Message", placeholder="message")

    with gr.Row():
      clear_btn = gr.Button("Clear")
      submit_btn = gr.Button("send")

    gr.HTML("""<p align="right">made by 문태서</p>""")
    gr.HTML("""<p align="right"><a href='https://github.com/Taeseo06/SchoolHelperChatbot-Gemma2B-Finetuning'>프로젝트 깃허브</a></p>""")

    def respond(system_prompt, main_input, history):
        response = gemma_chat(main_input, history, system_prompt)
        history.append((main_input, response))
        return "", history

    submit_btn.click(respond, inputs=[system_prompt, main_input, chatbox], outputs=[main_input,chatbox])
    clear_btn.click(clear_fn, None, [chatbox])

demo.launch(debug=True)


# 가장 답변의 답변의 정확도가 높았던 시스템 프롬프트:
'''
You are the School Life chatbot.
Please answer the questions below in Korean.
'''

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://806ed5bfa75f8e0808.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]